In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from torch.utils import data
import math

import os
#print(os.listdir("../input/nvidiaapex/repository/NVIDIA-apex-39e153a"))
#print(os.listdir("../input/glove-global-vectors-for-word-representation"))
#print(os.listdir("../input/jigsaw-unintended-bias-in-toxicity-classification"))
#print(os.listdir("../input/fasttext-crawl-300d-2m"))

# Any results you write to the current directory are saved as output.

In [2]:
# Installing Nvidia Apex
#This line is for Kaggle:
#! pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ../input/nvidiaapex/repository/NVIDIA-apex-39e153a

#On Cicero it should be this:
! pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ../input/NVIDIA-apex-39e153a

/home/curtis/miniconda3/lib/python3.7/site-packages/pip/_internal/commands/install.py:244: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-esy63wux
Created temporary directory: /tmp/pip-req-tracker-q2h04sdm
Created requirements tracker '/tmp/pip-req-tracker-q2h04sdm'
Created temporary directory: /tmp/pip-install-xxpbb036
Processing /home/curtis/Desktop/kaggle/jigsaw/input/NVIDIA-apex-39e153a
  Created temporary directory: /tmp/pip-req-build-n_xar9df
  Added file:///home/curtis/Desktop/kaggle/jigsaw/input/NVIDIA-apex-39e153a to build tracker '/tmp/pip-req-tracker-q2h04sdm'
    Running setup.py (path:/tmp/pip-req-build-n_xar9df/setup.py) egg_info for package from file:///home/curtis/Desktop/kaggle/jigsaw/input/NVIDIA-apex-39e153a
    Running command python setup.py egg_info
    torch.__version__  =  1.1.0
    running e

    g++ -pthread -shared -B /home/curtis/miniconda3/compiler_compat -L/home/curtis/miniconda3/lib -Wl,-rpath=/home/curtis/miniconda3/lib -Wl,--no-as-needed -Wl,--sysroot=/ build/temp.linux-x86_64-3.7/csrc/flatten_unflatten.o -o build/lib.linux-x86_64-3.7/apex_C.cpython-37m-x86_64-linux-gnu.so
    building 'amp_C' extension
    gcc -pthread -B /home/curtis/miniconda3/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/home/curtis/miniconda3/lib/python3.7/site-packages/torch/include -I/home/curtis/miniconda3/lib/python3.7/site-packages/torch/include/torch/csrc/api/include -I/home/curtis/miniconda3/lib/python3.7/site-packages/torch/include/TH -I/home/curtis/miniconda3/lib/python3.7/site-packages/torch/include/THC -I/usr/local/cuda/include -I/home/curtis/miniconda3/include/python3.7m -c csrc/amp_C_frontend.cpp -o build/temp.linux-x86_64-3.7/csrc/amp_C_frontend.o -O3 -DTORCH_API_INCLUDE_EXTENSION_H -DTORCH_EXTENSION_NAME=amp_C -D_GLIBCXX

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import datetime
import pkg_resources
import seaborn as sns
import time
import scipy.stats as stats
import gc
import re
import operator 
import sys
from sklearn import metrics
from sklearn import model_selection
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from nltk.stem import PorterStemmer
from sklearn.metrics import roc_auc_score
%load_ext autoreload
%autoreload 2
%matplotlib inline
from tqdm import tqdm, tqdm_notebook
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
from apex import amp
import shutil
from sklearn.model_selection import StratifiedKFold

ModuleNotFoundError: No module named 'apex'

In [ ]:
device=torch.device('cuda')

In [ ]:
MAX_SEQUENCE_LENGTH = 220
SEED = 1234
EPOCHS = 1
Data_dir="../input"
Input_dir = "../input"
WORK_DIR = "../working/"
TOXICITY_COLUMN = 'target'

In [ ]:
# Add the Bart Pytorch repo to the PATH
# using files from: https://github.com/huggingface/pytorch-pretrained-BERT

#On Kaggle it is:
#package_dir_a = "../input/ppbert/pytorch-pretrained-bert/pytorch-pretrained-BERT"

#Oncicero it is:
package_dir_a = "../input/pytorch-pretrained-BERT"

sys.path.insert(0, package_dir_a)

from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification,BertAdam

In [ ]:
# Translate model from tensorflow to pytorch

#On Kaggle:
#BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/'

#On Cicero:
BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_L-12_H-768_A-12/'

convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
    BERT_MODEL_PATH + 'bert_model.ckpt',
BERT_MODEL_PATH + 'bert_config.json',
WORK_DIR + 'pytorch_model.bin')

shutil.copyfile(BERT_MODEL_PATH + 'bert_config.json', WORK_DIR + 'bert_config.json')

In [ ]:
os.listdir("../working")

In [ ]:
# This is the Bert configuration file
from pytorch_pretrained_bert import BertConfig

#On Kaggle:
#bert_config = BertConfig('../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/'+'bert_config.json')

#On Cicero:
bert_config = BertConfig('../input/bert-pretrained-models/uncased_L-12_H-768_A-12/'+'bert_config.json')

In [ ]:
# Converting the lines to BERT format
# Thanks to https://www.kaggle.com/httpwwwfszyc/bert-in-keras-taming
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm_notebook(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            first = int(math.ceil(max_seq_length * 0.25)) #first 25%
            last = max_seq_length - first
            tokens_first = tokens_a[:first]
            tokens_last = tokens_a[-last:]
            tokens_a = tokens_first + tokens_last
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
       
        all_tokens.append(one_token)
    print("Number of sequences longer: ", longer)
    return np.array(all_tokens)

In [ ]:
%%time
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)
train_df = pd.read_csv(os.path.join(Data_dir,"train.csv"))

train_df = train_df.sample(len(train_df),random_state=SEED)
print('loaded %d records' % len(train_df))

# Make sure all comment_text values are strings
train_df['comment_text'] = train_df['comment_text'].astype(str) 

sequences = convert_lines(train_df["comment_text"].fillna("DUMMY_VALUE"),MAX_SEQUENCE_LENGTH,tokenizer)
train_df=train_df.fillna(0)
# List all identities
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
y_columns=['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat', 'sexual_explicit']
difficult_cols = ['muslim', 'white', 'homosexual_gay_or_lesbian', 'black']

print("wtf1")

y_aux_train = train_df[identity_columns]

# Overall
weights = np.ones((len(train_df),)) * 0.25
# Subgroup
weights += (train_df[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) * 0.25

# Background Positive, Subgroup Negative
weights += (( (train_df['target'].values>=0.5).astype(bool).astype(np.int) +
   (train_df[identity_columns].fillna(0).values<0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) * 0.25
# Background Negative, Subgroup Positive
weights += (( (train_df['target'].values<0.5).astype(bool).astype(np.int) +
   (train_df[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) * 0.25
loss_weight = 1.0 / weights.mean()

print("wtf2")

train_df = train_df.drop(['comment_text'],axis=1)
# convert target to 0,1
#train_df['target']=(train_df['target']>=0.5).astype(float)
print("wtf3")
#train_df['target'] = np.vstack([(train_df['target'].values>=0.5).astype(float),weights]).T
train_df['target']=(train_df['target']>=0.5).astype(float)
print("Target: ", train_df["target"])

In [ ]:
X = sequences
y = train_df[y_columns].values

In [ ]:
x_train_torch = torch.tensor(X,dtype=torch.long)
y_train_torch = torch.tensor(y,dtype=torch.float)
full_train_dataset = torch.utils.data.TensorDataset(x_train_torch, y_train_torch)

In [ ]:
class LenMatchBatchSampler(data.BatchSampler):
    def __iter__(self):

        buckets = [[]] * 100
        yielded = 0

        for idx in self.sampler:
            count_zeros = torch.sum(self.sampler.data_source[idx][0] == 0)
            count_zeros = int(count_zeros / 64) 
            if len(buckets[count_zeros]) == 0:  buckets[count_zeros] = []

            buckets[count_zeros].append(idx)

            if len(buckets[count_zeros]) == self.batch_size:
                batch = list(buckets[count_zeros])
                yield batch
                yielded += 1
                buckets[count_zeros] = []

        batch = []
        leftover = [idx for bucket in buckets for idx in bucket]

        for idx in leftover:
            batch.append(idx)
            if len(batch) == self.batch_size:
                yielded += 1
                yield batch
                batch = []

        if len(batch) > 0 and not self.drop_last:
            yielded += 1
            yield batch

        assert len(self) == yielded, "produced an inccorect number of batches. expected %i, but yielded %i" %(len(self), yielded)

def trim_tensors(tsrs):
    max_len = torch.max(torch.sum( (tsrs[0] != 0  ), 1))
    if max_len > 2: 
        tsrs = [tsr[:, :max_len] for tsr in tsrs]
    return tsrs 

In [ ]:
def custom_loss(data, targets):
    ''' Define custom loss function for weighted BCE on 'target' column '''
    #print("Data shape: ", data.shape)
    #print("Target shape: ", targets.shape)
    bce_loss_1 = nn.BCEWithLogitsLoss(weight=targets[:,1:2])(data[:,:1],targets[:,:1])
    bce_loss_2 = nn.BCEWithLogitsLoss()(data[:,1:],targets[:,1:])
    return (bce_loss_1 * loss_weight) + bce_loss_2

In [ ]:
random_state = 42
NUM_SPLITS = 5
skf = StratifiedKFold(n_splits = NUM_SPLITS, shuffle=True, random_state=random_state)

In [ ]:
output_model_file = "bert_pytorch.bin"

lr=2e-5
batch_size = 32
accumulation_steps=2
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

model = BertForSequenceClassification.from_pretrained("../working",cache_dir=None,num_labels=len(y_columns))
model.zero_grad()
model = model.to(device)
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

tq = tqdm_notebook(range(EPOCHS))
#nm = tqdm_notebook(range(NUM_MOLDS))

bert_preds = np.zeros((len(full_train_dataset)))
for fold, (trn_idxs, val_idxs) in enumerate(skf.split(x_train_torch,y_train_torch[:,0])):
    train_dataset = full_train_dataset[trn_idxs]
    validation_x = full_train_dataset[val_idxs]
    
    num_train_optimization_steps = int(EPOCHS*len(train_dataset)/batch_size/accumulation_steps)

    optimizer = BertAdam(optimizer_grouped_parameters,
                         lr=lr,
                         warmup=0.05,
                         t_total=num_train_optimization_steps)

    model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
    print(model)
    model=model.train()
    print(model)
    print("DOUBLE CHECK MODEL IS NOT NONE")

    for epoch in tq:
        avg_loss = 0.
        avg_accuracy = 0.
        lossf=None

        ran_sampler = data.RandomSampler(train_dataset)
        len_sampler = LenMatchBatchSampler(ran_sampler, batch_size = 32, drop_last = False)
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_sampler = len_sampler) 

        tk0 = tqdm_notebook(enumerate(train_loader),total=len(train_loader),leave=False)
        optimizer.zero_grad() 
        for i, batch in tk0:
            #print(batch)
            tsrs = trim_tensors(batch)
            x_batch, y_batch = tuple(t.to(device) for t in tsrs)

            y_pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
            loss =  F.binary_cross_entropy_with_logits(y_pred,y_batch.to(device))
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
            if (i+1) % accumulation_steps == 0:             # Wait for several backward steps
                optimizer.step()                            # Now we can do an optimizer step
                optimizer.zero_grad()
            if lossf:
                lossf = 0.98*lossf+0.02*loss.item()
            else:
                lossf = loss.item()
            tk0.set_postfix(loss = lossf)
            avg_loss += loss.item() / len(train_loader)
            avg_accuracy += torch.mean(((torch.sigmoid(y_pred[:,0])>=0.5) == (y_batch[:,0]>=0.5).to(device)).to(torch.float) ).item()/len(train_loader)
        tq.set_postfix(avg_loss=avg_loss,avg_accuracy=avg_accuracy)

    # predicting on the validation set
    model.eval()
    validation_preds = np.zeros((len(validation_x)))
    validation = torch.utils.data.TensorDataset(torch.tensor(validation_x, dtype=torch.long))
    validation_loader = torch.utils.data.DataLoader(validation, batch_size=32, shuffle=False)
    tk0 = tqdm(validation_loader)
    for i, (x_batch,) in enumerate(tk0):
        pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
        validation_preds[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()
    bert_preds[val_idxs] = validation_preds
    
    # saving a backup of the current fold's predictions
    np.save('bert_stacking_train_preds_' +  str(fold)+ '.npy', new_train)
    
    # save the model
    model_name = "bert_" + str(fold) + ".bin"
    torch.save(model.state_dict(), model_name)

In [ ]:
# once all the folds are done bert_preds should have the preds from all the validations

new_train = np.c_[bert_preds, y]
np.save('bert_stacking_train.npy', new_train) # save
#new_num_arr = np.load('data.npy') # load

"""bert_train = pd.DataFrame({bert_preds
                           
bert_train = pd.DataFrame.from_dict({
    'id': test['id'],
    'prediction': np.mean(all_test_preds, axis=0)[:, 0]
})

submission.to_hdf('submission.csv', index=False)"""